In [1]:
import pickle
import os
import time
import numpy as np
import polychrom

from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
from polychrom.simulation import Simulation
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file

import simtk.openmm 
import os 
import shutil


import warnings
import h5py 
import glob

## Defining simulation parameters

In [3]:
# -------defining parameters----------
#  -- basic loop extrusion parameters

folder = "trajectory"

myfile = h5py.File("trajectory/LEFPositions.h5", mode='r')

N = myfile.attrs["N"]
LEFNum = myfile.attrs["LEFNum"]
LEFpositions = myfile["positions"]

Nframes = LEFpositions.shape[0]

    
steps = 250   # MD steps per step of cohesin
stiff = 1
dens = 0.1
box = (N / dens) ** 0.33  # density = 0.1.
data = grow_cubic(N, int(box) - 2)  # creates a compact conformation 
block = 0  # starting block 

# new parameters because some things changed 
saveEveryBlocks = 10   # save every 10 blocks (saving every block is now too much almost)
restartSimulationEveryBlocks = 100

# parameters for smc bonds
smcBondWiggleDist = 0.2
smcBondDist = 0.5

# assertions for easy managing code below 
assert (Nframes % restartSimulationEveryBlocks) == 0 
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal  = (Nframes) // restartSimulationEveryBlocks

## Bond updating functions (for increased speed)

In [4]:
class bondUpdater(object):

    def __init__(self, LEFpositions):
        """
        :param smcTransObject: smc translocator object to work with
        """
        self.LEFpositions = LEFpositions
        self.curtime  = 0
        self.allBonds = []

    def setParams(self, activeParamDict, inactiveParamDict):
        """
        A method to set parameters for bonds.
        It is a separate method because you may want to have a Simulation object already existing

        :param activeParamDict: a dict (argument:value) of addBond arguments for active bonds
        :param inactiveParamDict:  a dict (argument:value) of addBond arguments for inactive bonds

        """
        self.activeParamDict = activeParamDict
        self.inactiveParamDict = inactiveParamDict


    def setup(self, bondForce,  blocks=100, smcStepsPerBlock=1):
        """
        A method that milks smcTranslocator object
        and creates a set of unique bonds, etc.

        :param bondForce: a bondforce object (new after simulation restart!)
        :param blocks: number of blocks to precalculate
        :param smcStepsPerBlock: number of smcTranslocator steps per block
        :return:
        """


        if len(self.allBonds) != 0:
            raise ValueError("Not all bonds were used; {0} sets left".format(len(self.allBonds)))

        self.bondForce = bondForce

        #precalculating all bonds
        allBonds = []
        
        loaded_positions  = self.LEFpositions[self.curtime : self.curtime+blocks]
        allBonds = [[(int(loaded_positions[i, j, 0]), int(loaded_positions[i, j, 1])) 
                        for j in range(loaded_positions.shape[1])] for i in range(blocks)]

        self.allBonds = allBonds
        self.uniqueBonds = list(set(sum(allBonds, [])))

        #adding forces and getting bond indices
        self.bondInds = []
        self.curBonds = allBonds.pop(0)

        for bond in self.uniqueBonds:
            paramset = self.activeParamDict if (bond in self.curBonds) else self.inactiveParamDict
            ind = bondForce.addBond(bond[0], bond[1], **paramset) # changed from addBond
            self.bondInds.append(ind)
        self.bondToInd = {i:j for i,j in zip(self.uniqueBonds, self.bondInds)}
        
        self.curtime += blocks 
        
        return self.curBonds,[]


    def step(self, context, verbose=False):
        """
        Update the bonds to the next step.
        It sets bonds for you automatically!
        :param context:  context
        :return: (current bonds, previous step bonds); just for reference
        """
        if len(self.allBonds) == 0:
            raise ValueError("No bonds left to run; you should restart simulation and run setup  again")

        pastBonds = self.curBonds
        self.curBonds = self.allBonds.pop(0)  # getting current bonds
        bondsRemove = [i for i in pastBonds if i not in self.curBonds]
        bondsAdd = [i for i in self.curBonds if i not in pastBonds]
        bondsStay = [i for i in pastBonds if i in self.curBonds]
        if verbose:
            print("{0} bonds stay, {1} new bonds, {2} bonds removed".format(len(bondsStay),
                                                                            len(bondsAdd), len(bondsRemove)))
        bondsToChange = bondsAdd + bondsRemove
        bondsIsAdd = [True] * len(bondsAdd) + [False] * len(bondsRemove)
        for bond, isAdd in zip(bondsToChange, bondsIsAdd):
            ind = self.bondToInd[bond]
            paramset = self.activeParamDict if isAdd else self.inactiveParamDict
            self.bondForce.setBondParameters(ind, bond[0], bond[1], **paramset)  # actually updating bonds
        self.bondForce.updateParametersInContext(context)  # now run this to update things in the context
        return self.curBonds, pastBonds

## Run simulation 

In [ ]:
milker = bondUpdater(LEFpositions)

reporter = HDF5Reporter(folder=folder, max_data_length=100, overwrite=True, blocks_only=False)

for iteration in range(simInitsTotal):
    
    # simulation parameters are defined below 
    a = Simulation(
            platform="cuda",
            integrator="variableLangevin", 
            error_tol=0.01, 
            GPU = "0", 
            collision_rate=0.03, 
            N = len(data),
            reporters=[reporter],
            PBCbox=[box, box, box],
            precision="mixed")  # timestep not necessary for variableLangevin

    ############################## New code ##############################
    a.set_data(data)  # loads a polymer, puts a center of mass at zero
    
    a.add_force(
        forcekits.polymer_chains(
            a,
            chains=[(0, None, None)],

                # By default the library assumes you have one polymer chain
                # If you want to make it a ring, or more than one chain, use self.setChains
                # self.setChains([(0,50,1),(50,None,0)]) will set a 50-monomer ring and a chain from monomer 50 to the end

            bond_force_func=forces.harmonic_bonds,
            bond_force_kwargs={
                'bondLength':1.0,
                'bondWiggleDistance':0.1, # Bond distance will fluctuate +- 0.05 on average
             },

            angle_force_func=forces.angle_force,
            angle_force_kwargs={
                'k':0.05
                # K is more or less arbitrary, k=4 corresponds to presistence length of 4,
                # k=1.5 is recommended to make polymer realistically flexible; k=8 is very stiff
            },

            nonbonded_force_func=forces.polynomial_repulsive,
            nonbonded_force_kwargs={
                'trunc':1.5, # this will let chains cross sometimes
                'radiusMult':1.05, # this is from old code
                #'trunc':10.0, # this will resolve chain crossings and will not let chain cross anymore
            },

            except_bonds=True,
             
        )
    )
    
    # ------------ initializing milker; adding bonds ---------
    # copied from addBond
    kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
    bondDist = smcBondDist * a.length_scale

    activeParams = {"length":bondDist,"k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    milker.setParams(activeParams, inactiveParams)
     
    # this step actually puts all bonds in and sets first bonds to be what they should be
    milker.setup(bondForce=a.force_dict['harmonic_bonds'],
                blocks=restartSimulationEveryBlocks)

    # If your simulation does not start, consider using energy minimization below
    if iteration==0:
        a.local_energy_minimization() 
    else:
        a._apply_forces()
    
    for i in range(restartSimulationEveryBlocks):        
        if i % saveEveryBlocks == (saveEveryBlocks - 1):  
            a.do_block(steps=steps)
        else:
            a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)
        if i < restartSimulationEveryBlocks - 1: 
            curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
    data = a.get_data()  # save data and step, and delete the simulation
    del a
    
    reporter.blocks_only = True  # Write output hdf5-files only for blocks

    time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)
    
reporter.dump_data()

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force polynomial_repulsive 2


Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 0.404798
INFO:root:before minimization eK=1.4998521514336989, eP=0.4047984224871978, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 0.066671
INFO:root:after minimization eK=1.4998521514336989, eP=0.029944362223079574, time=0.0 ps
INFO:root:block    0 pos[1]=[39.9 30.4 34.9] dr=12.54 t=433.5ps kin=1.51 pot=0.70 Rg=28.942 dt=173.3fs dx=47.49pm 
INFO:root:block    1 pos[1]=[31.9 34.5 34.5] dr=6.31 t=866.9ps kin=1.51 pot=0.71 Rg=30.864 dt=173.3fs dx=47.56pm 
INFO:root:block    2 pos[1]=[44.5 30.6 35.7] dr=6.20 t=1300.2ps kin=1.51 pot=0.69 Rg=32.105 dt=173.3fs dx=47.50pm 
INFO:root:block    3 pos[1]=[35.9 24.2 37.0] dr=6.15 t=1733.5ps kin=1.51 pot=0.70 Rg=32.924 dt=173.3fs dx=47.64pm 
INFO:root:block    4 pos[1]=[37.6 25.4 27.0] dr=6.12 t=2166.9ps kin=1.50 pot=0.70 Rg=33.404 dt=173.3fs dx=47.38pm 
INFO:root:block    5 pos[1]=[41.5 31.5 29.1] dr=6.19 t=2600.2ps kin=1.50 pot=0.69 Rg=33.855 dt=173.3fs dx=47.43pm 
INFO:root:block    6

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 0.707292
INFO:root:block    0 pos[1]=[44.9 33.1 30.6] dr=6.17 t=429.6ps kin=1.51 pot=0.69 Rg=34.539 dt=171.8fs dx=47.09pm 
INFO:root:block    1 pos[1]=[37.7 24.6 36.5] dr=6.16 t=859.3ps kin=1.51 pot=0.70 Rg=34.632 dt=171.8fs dx=47.09pm 
INFO:root:block    2 pos[1]=[48.7 25.0 38.9] dr=6.21 t=1288.9ps kin=1.50 pot=0.70 Rg=34.649 dt=171.8fs dx=46.97pm 
INFO:root:block    3 pos[1]=[49.3 22.8 34.8] dr=6.10 t=1718.5ps kin=1.51 pot=0.70 Rg=34.634 dt=171.8fs dx=47.11pm 
INFO:root:block    4 pos[1]=[56.4 26.5 28.3] dr=6.10 t=2148.1ps kin=1.51 pot=0.69 Rg=34.618 dt=171.8fs dx=47.12pm 
INFO:root:block    5 pos[1]=[49.1 14.7 23.2] dr=6.14 t=2577.7ps kin=1.51 pot=0.69 Rg=34.583 dt=171.8fs dx=47.20pm 
INFO:root:block    6 pos[1]=[43.9 8.6 27.1] dr=6.16 t=3007.3ps kin=1.50 pot=0.70 Rg=34.615 dt=171.8fs dx=47.08pm 
INFO:root:block    7 pos[1]=[45.8 22.9 22.4] dr=6.14 t=3436.9ps kin=1.49 pot=0.70 Rg=34.878 dt=171.8fs dx=46.90pm 
INFO:root:block    8 pos[1

...

INFO:root:Particles loaded. Potential energy is 0.715854
INFO:root:block    0 pos[1]=[47.8 13.2 32.0] dr=6.05 t=429.2ps kin=1.51 pot=0.69 Rg=34.775 dt=171.7fs dx=47.11pm 
INFO:root:block    1 pos[1]=[55.0 9.7 28.8] dr=6.09 t=858.4ps kin=1.51 pot=0.69 Rg=34.891 dt=171.7fs dx=47.09pm 
INFO:root:block    2 pos[1]=[58.5 9.3 27.4] dr=6.11 t=1287.6ps kin=1.52 pot=0.69 Rg=34.977 dt=171.7fs dx=47.24pm 
INFO:root:block    3 pos[1]=[46.0 7.0 30.5] dr=6.04 t=1716.8ps kin=1.51 pot=0.68 Rg=34.893 dt=171.7fs dx=47.08pm 
INFO:root:block    4 pos[1]=[53.2 11.7 36.5] dr=6.01 t=2146.0ps kin=1.52 pot=0.70 Rg=34.953 dt=171.7fs dx=47.22pm 
INFO:root:block    5 pos[1]=[48.3 16.8 37.5] dr=6.10 t=2575.2ps kin=1.50 pot=0.70 Rg=35.036 dt=171.7fs dx=47.04pm 
INFO:root:block    6 pos[1]=[49.3 16.2 38.1] dr=6.18 t=3004.4ps kin=1.50 pot=0.69 Rg=35.017 dt=171.7fs dx=47.01pm 
INFO:root:block    7 pos[1]=[47.6 10.1 33.2] dr=6.12 t=3433.6ps kin=1.50 pot=0.69 Rg=35.135 dt=171.7fs dx=46.97pm 
INFO:root:block    8 pos[1]=

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 0.717262
INFO:root:block    0 pos[1]=[55.3 10.7 34.4] dr=6.07 t=428.7ps kin=1.51 pot=0.69 Rg=35.094 dt=171.5fs dx=47.11pm 
INFO:root:block    1 pos[1]=[45.3 12.9 26.9] dr=6.15 t=857.3ps kin=1.51 pot=0.69 Rg=35.018 dt=171.5fs dx=47.02pm 
INFO:root:block    2 pos[1]=[41.9 13.9 27.9] dr=6.19 t=1286.0ps kin=1.50 pot=0.69 Rg=34.943 dt=171.5fs dx=46.87pm 
INFO:root:block    3 pos[1]=[36.9 8.5 26.0] dr=6.13 t=1714.7ps kin=1.50 pot=0.68 Rg=34.918 dt=171.5fs dx=46.95pm 
INFO:root:block    4 pos[1]=[39.5 3.5 25.6] dr=6.18 t=2143.3ps kin=1.51 pot=0.70 Rg=34.782 dt=171.5fs dx=47.01pm 
INFO:root:block    5 pos[1]=[34.2 7.3 26.6] dr=6.07 t=2572.0ps kin=1.50 pot=0.70 Rg=34.950 dt=171.5fs dx=46.92pm 
INFO:root:block    6 pos[1]=[41.2 3.4 23.8] dr=6.12 t=3000.6ps kin=1.50 pot=0.69 Rg=34.931 dt=171.5fs dx=46.93pm 
INFO:root:block    7 pos[1]=[51.1 4.2 24.3] dr=6.11 t=3429.3ps kin=1.50 pot=0.69 Rg=34.931 dt=171.5fs dx=46.93pm 
INFO:root:block    8 pos[1]=[4

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 0.716039
INFO:root:block    0 pos[1]=[40.8 12.7 32.0] dr=6.07 t=429.4ps kin=1.51 pot=0.69 Rg=34.927 dt=171.8fs dx=47.20pm 
INFO:root:block    1 pos[1]=[45.0 13.4 29.3] dr=6.05 t=858.8ps kin=1.50 pot=0.68 Rg=35.068 dt=171.8fs dx=47.03pm 
INFO:root:block    2 pos[1]=[44.7 10.4 33.0] dr=6.10 t=1288.2ps kin=1.51 pot=0.69 Rg=35.176 dt=171.8fs dx=47.21pm 
INFO:root:block    3 pos[1]=[53.6 5.8 31.7] dr=6.05 t=1717.6ps kin=1.51 pot=0.69 Rg=35.087 dt=171.8fs dx=47.11pm 
INFO:root:block    4 pos[1]=[50.7 9.6 31.3] dr=6.15 t=2147.0ps kin=1.50 pot=0.69 Rg=35.247 dt=171.8fs dx=47.03pm 
INFO:root:block    5 pos[1]=[49.3 2.8 37.0] dr=6.04 t=2576.4ps kin=1.51 pot=0.69 Rg=35.210 dt=171.8fs dx=47.11pm 
